In [1]:
import numpy as np
import pandas as pd
from scipy.io import mmread
import os
import glob
import anndata
import tqdm
import zarr
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sp
import zarr

from dredFISH.Utils import basicu
from dredFISH.Utils import powerplots

- merge marker genes (types ABC)
- feed into NN design

In [2]:
pth = '/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P38_nr_1_a/filtered_feature_bc_matrix'
f = os.path.join(pth, 'counts.h5ad')
adata = anndata.read(f)
adata.obs.index = np.char.add(adata.obs.index.values.astype(str), "-P38_1a-P38")
adata

AnnData object with n_obs × n_vars = 10357 × 53801
    var: 'id', 'name'

In [3]:
cov = np.asarray(adata.X.sum(axis=1)).reshape(-1,)
medcov = np.median(cov)

adata.obs['cov'] = cov
adata.obs['covfactor'] = cov/medcov

In [4]:
adata.obs

,cov,covfactor
AAACCCAAGGCCCAAA-1-P38_1a-P38,1543.0,0.313363
AAACCCAAGTGACACG-1-P38_1a-P38,8590.0,1.744517
AAACCCACAGACCGCT-1-P38_1a-P38,2300.0,0.467100
AAACCCACAGGCTTGC-1-P38_1a-P38,519.0,0.105402
AAACCCACATCCTGTC-1-P38_1a-P38,542.0,0.110073
...,...,...
TTTGTTGGTATCACGT-1-P38_1a-P38,1737.0,0.352762
TTTGTTGGTCACTCGG-1-P38_1a-P38,8944.0,1.816409
TTTGTTGGTCCAGCAC-1-P38_1a-P38,2490.0,0.505686
TTTGTTGGTTATGACC-1-P38_1a-P38,1367.0,0.277620


In [5]:
data_dir = '/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/'

In [6]:
f = '../data_dump/candidate_genes_vincent_0503_v2.csv'
df0 = pd.read_csv(f)
df0 = df0.sort_values(['Subclass', 'Type/Gradient'])
df0 = df0[df0['Gene']!='Neurod1']
df0 = df0.set_index('Gene')
df0

,Subclass,Type/Gradient,Function,Priority,Allen marker
Gene,,,,,
Baz1a,L2/3,A<B>C,TF,***,NaN
Bdnf,L2/3,A<B>C,CAM,***,NaN
Cdh4,L2/3,A<B>C,CAM,***,NaN
Epha3,L2/3,A<B>C,CAM,***,NaN
Grm8,L2/3,A<B>C,IC,NaN,NaN
...,...,...,...,...,...
Rrad,L2/3,NaN,other,NaN,x
S100a6,L2/3,NaN,other,NaN,x
Chrm2,"L2/3, L4",C>B=A,IC,***,NaN


In [7]:
df0['Type/Gradient'].unique(), df0['Subclass'].unique()

(array(['A<B>C', 'A>B=C', 'A>B>C', 'A>C>B', 'C>A>B', 'C>B=A', 'C>B>A',
        'all', nan], dtype=object),
 array(['L2/3', 'L2/3, L4', 'L4'], dtype=object))

In [8]:
genes = df0.index.values
allen_genes = df0[~df0['Allen marker'].isnull()].index.values
allen_genes.shape

(14,)

In [9]:
f = os.path.join(data_dir, 'genes_lasso_all83_May13.csv')
g1 = pd.read_csv(f, index_col=0)
print(g1.shape)

f = os.path.join(data_dir, 'genes_lasso_nonallen_May13.csv')
g2 = pd.read_csv(f, index_col=0)
print(g2.shape)

(16, 10)
(17, 10)


In [10]:
%%time
adatasub = anndata.read('../data_dump/P38_glut_genes_v3.h5ad')
adatasub

CPU times: user 186 ms, sys: 706 ms, total: 892 ms
Wall time: 894 ms


AnnData object with n_obs × n_vars = 5837 × 83
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [11]:
adatasub.obs['sample'].unique()

['P38_1a', 'P38_2a', 'P38_2b']
Categories (3, object): ['P38_1a', 'P38_2a', 'P38_2b']

In [12]:
tmp = adata[adatasub[adatasub.obs['sample']=='P38_1a'].obs.index].copy()
unq, inv, cnt = np.unique(tmp.var['name'].values, return_counts=True, return_inverse=True)
tmp = tmp[:, cnt[inv] == 1]
genes_idx = basicu.get_index_from_array(tmp.var['name'].values.astype(str), df0.index.values.astype(str))
tmp = tmp[:, genes_idx]
tmp.var = tmp.var.set_index('name')
tmp

/home/fangming/anaconda3/envs/routine/lib/python3.10/site-packages/anndata/_core/anndata.py:801: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['Baz1a', 'Bdnf', 'Cdh4', 'Epha3', 'Grm8']

    Inferred to be: categorical

  value_idx = self._prep_dim_index(value.index, attr)


AnnData object with n_obs × n_vars = 1549 × 83
    obs: 'cov', 'covfactor'
    var: 'id'

In [13]:
mat = np.asarray(tmp.X.todense())
matnorm = mat/tmp.obs['covfactor'].values.reshape(-1,1)
mat.shape, matnorm.shape

((1549, 83), (1549, 83))

In [14]:
tmp.layers['normed'] = matnorm
tmp.obs = tmp.obs.join(adatasub.obs)
tmp.X = np.asarray(tmp.X.todense())

In [16]:
tmp

AnnData object with n_obs × n_vars = 1549 × 83
    obs: 'cov', 'covfactor', 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'id'
    layers: 'normed'

In [17]:
tmp.obs

,cov,covfactor,batch,n_genes,percent_mito,n_counts,leiden,Doublet,Doublet Score,cluster,Class_broad,sample,Age,subclass,layer,Type,Subclass
AAACCCAAGTGACACG-1-P38_1a-P38,8590.0,1.744517,P38,2931,0.001048,8590.0,0,False,0.010833,L2/3_B,Excitatory,P38_1a,P38,L2/3,L2/3/4,L2/3_B,L2/3
AAACGCTAGTACAGAT-1-P38_1a-P38,13659.0,2.773964,P38,4451,0.000952,13659.0,6,False,0.061252,L4_A,Excitatory,P38_1a,P38,L4,L2/3/4,L4_C,L4
AAACGCTAGTGAGGCT-1-P38_1a-P38,16269.0,3.304021,P38,4543,0.001475,16269.0,6,False,0.048147,L4_A,Excitatory,P38_1a,P38,L4,L2/3/4,L4_C,L4
AAACGCTGTATGGAAT-1-P38_1a-P38,6618.0,1.344029,P38,2678,0.001813,6618.0,3,False,0.013447,L4_B,Excitatory,P38_1a,P38,L4,L2/3/4,L4_B,L4
AAAGAACAGTAGTCCT-1-P38_1a-P38,12608.0,2.560520,P38,3781,0.000159,12607.0,6,False,0.029642,L4_A,Excitatory,P38_1a,P38,L4,L2/3/4,L4_C,L4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGAGCAGATGCGA-1-P38_1a-P38,5868.0,1.191714,P38,2276,0.000511,5868.0,3,False,0.017461,L4_B,Excitatory,P38_1a,P38,L4,L2/3/4,L4_B,L4
TTTGGTTAGGACACTG-1-P38_1a-P38,17776.0,3.610073,P38,4953,0.000338,17775.0,6,False,0.035124,L4_A,Excitatory,P38_1a,P38,L4,L2/3/4,L4_C,L4
TTTGGTTCACTTTATC-1-P38_1a-P38,15758.0,3.200244,P38,4399,0.000127,15758.0,0,False,0.061252,L2/3_B,Excitatory,P38_1a,P38,L2/3,L2/3/4,L2/3_B,L2/3
TTTGTTGCAGGTGAGT-1-P38_1a-P38,6636.0,1.347685,P38,2655,0.000753,6636.0,1,False,0.034145,L4_C,Excitatory,P38_1a,P38,L4,L2/3/4,L4_A,L4


In [19]:
tmp.X

array([[0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 4., 1., ..., 0., 5., 0.],
       [0., 1., 0., ..., 6., 1., 4.],
       [9., 6., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
tmp.layers['normed']

array([[0.       , 0.       , 0.       , ..., 0.       , 1.1464494,
        0.       ],
       [0.       , 0.       , 0.3604949, ..., 0.       , 0.       ,
        0.3604949],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.3026615],
       ...,
       [0.       , 1.2499048, 0.3124762, ..., 0.       , 1.562381 ,
        0.       ],
       [0.       , 0.7420132, 0.       , ..., 4.4520793, 0.7420132,
        2.9680529],
       [4.9548297, 3.30322  , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [21]:
tmp

AnnData object with n_obs × n_vars = 1549 × 83
    obs: 'cov', 'covfactor', 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'id'
    layers: 'normed'

In [1]:
dat_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/'
output = os.path.join(dat_dir, 'rna', 'scrna_visctx_P38_test.zarr')
print(output)

NameError: name 'os' is not defined

In [4]:
%%time
chunksize = 10
cells = adata.obs.index.values
genes = adata.var.index.values
ncells = len(cells)
ngenes = len(genes)

# create a zarr file (dir)
z1 = zarr.open(
          output, 
          mode='w', 
    )
z1

CPU times: user 4.22 ms, sys: 1.51 ms, total: 5.73 ms
Wall time: 134 ms


<zarr.hierarchy.Group '/'>

In [5]:
%%time
# create count matrix
z1.create_dataset('counts', 
                  shape=(ncells, ngenes), 
                  chunks=(chunksize, None), 
                  dtype='i4',
                 )
z1['counts'][:] = adata.X

# create count matrix
z1.create_dataset('normed', 
                  shape=(ncells, ngenes), 
                  chunks=(chunksize, None), 
                  dtype='i4',
                 )
z1['normed'][:] = adata.layers['normed']

CPU times: user 1min 40s, sys: 1min 26s, total: 3min 7s
Wall time: 3min 23s


In [6]:
# create labels
z1.create_dataset(
    'l1_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = adata.obs['class_label']
z1['l1_code'] = a.cat.codes.values
z1['l1_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l2_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = adata.obs['neighborhood_label']
z1['l2_code'] = a.cat.codes.values
z1['l2_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l3_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = adata.obs['subclass_label']
z1['l3_code'] = a.cat.codes.values
z1['l3_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l5_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = adata.obs['cluster_label']
z1['l5_code'] = a.cat.codes.values
z1['l5_cat'] = a.cat.categories.values.astype(str)

In [11]:
%%time
y = z1['counts'].oindex[975,:]
y, y.shape

CPU times: user 87 µs, sys: 8.16 ms, total: 8.25 ms
Wall time: 6.69 ms


(array([  0,   0, 242, ...,   0,   0,   0], dtype=int32), (45768,))

In [12]:
%%time
testidx = np.random.choice(ncells, 100)

CPU times: user 137 µs, sys: 142 µs, total: 279 µs
Wall time: 291 µs


In [13]:
%%time
z1['counts'].oindex[testidx,:].shape

CPU times: user 1.09 s, sys: 152 ms, total: 1.24 s
Wall time: 1.2 s


(100, 45768)

In [14]:
%%time
z1['counts'].oindex[testidx[0],:].shape

CPU times: user 4.6 ms, sys: 4.61 ms, total: 9.21 ms
Wall time: 5.65 ms


(45768,)

In [16]:
%%time
z1['l3_code'].oindex[testidx[0]].shape

CPU times: user 4.07 ms, sys: 4.07 ms, total: 8.14 ms
Wall time: 6.91 ms


()

# split train and test

In [71]:
def split_train_test(zarr_file, keys_copy, keys_split, frac=0.9, random_seed=None):
    """randomly select frac vs 1-frac samples into training and test (validation) set.
    Save them as separate zarr files
    """
    assert frac <= 1 and frac >= 0
    # the original zarr file
    z = zarr.open(zarr_file, 'r')
    size = len(z['counts'])
    
    path_train = zarr_file.replace('.zarr', '_train.zarr')
    path_test = zarr_file.replace('.zarr', '_test.zarr')
    print(f"{zarr_file} -> \n{path_train} and \n{path_test}\n")
    if random_seed: np.random.seed(random_seed)
    cond_train = np.random.rand(size) < frac
    ntrain = cond_train.sum()
    ntest = (~cond_train).sum()
    print(f"{size}, {ntrain} ({ntrain/size:.3f}), {ntest} ({ntest/size:.3f})")
    
    z_train = zarr.open(path_train, mode='w')
    z_test = zarr.open(path_test, mode='w')
    for key in keys_copy:
        z_train[key] = z[key]
        z_test[key] = z[key]

    for key in keys_split:
        # train
        if z[key].ndim == 1:
            chunksize = (10,)
            dat = z[key].oindex[cond_train]
        elif z[key].ndim == 2:
            chunksize = (10,None)
            dat = z[key].oindex[cond_train,:]
        else:
            raise ValueError('unimplemented')
            
        z_train.create_dataset(key, shape=dat.shape, chunks=chunksize)
        z_train[key][:] = dat
        
        # test
        if z[key].ndim == 1:
            chunksize = (10,)
            dat = z[key].oindex[~cond_train]
        elif z[key].ndim == 2:
            chunksize = (10,None)
            dat = z[key].oindex[~cond_train,:]
        else:
            raise ValueError('unimplemented')
            
        z_test.create_dataset(key, shape=dat.shape, chunks=chunksize)
        z_test[key][:] = dat
        
    return 

In [59]:
z = zarr.open(output, mode='r')
z, list(z.keys())

(<zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'])

In [60]:
#
keys_copy = ['num_probe_limit', 'l1_cat', 'l2_cat', 'l3_cat', 'l5_cat']
keys_split = [key for key in z.keys() if key not in keys_copy]
keys_copy, keys_split

(['num_probe_limit', 'l1_cat', 'l2_cat', 'l3_cat', 'l5_cat'],
 ['counts', 'l1_code', 'l2_code', 'l3_code', 'l5_code'])

In [72]:
split_train_test(output, keys_copy, keys_split, frac=0.9)

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3.zarr -> 
/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3_train.zarr and 
/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3_test.zarr

73347, 66051 (0.901), 7296 (0.099)


# test if we can load train and val as well as the original

In [89]:
f_org = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3.zarr'
f_trn = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3_train.zarr'
f_tst = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3_test.zarr'


In [85]:
z_trn = zarr.open(f_trn, mode='r')
z_tst = zarr.open(f_tst, mode='r')
z_trn, list(z_trn.keys()), z_tst, list(z_tst.keys())

(<zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'],
 <zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'])

In [87]:
z_trn['num_probe_limit'][:]

array([1, 1, 1, ..., 1, 1, 1])

In [81]:
%%time
y = z_trn['counts'].oindex[975,:]
y, y.shape

CPU times: user 6.89 ms, sys: 6.13 ms, total: 13 ms
Wall time: 7.75 ms


(array([  0.,   0., 365., ...,   0.,   0.,   0.]), (45768,))

In [82]:
%%time
y = z_tst['counts'].oindex[975,:]
y, y.shape

CPU times: user 950 µs, sys: 8.9 ms, total: 9.85 ms
Wall time: 5.57 ms


(array([  0.,   0., 112., ...,   0.,   0.,   0.]), (45768,))

# fix num probe limit (change r to a for read mode)

In [99]:
z_org = zarr.open(f_org, mode='r')
z_trn = zarr.open(f_trn, mode='r')
z_tst = zarr.open(f_tst, mode='r')
z_trn, list(z_trn.keys()), z_tst, list(z_tst.keys())

(<zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'],
 <zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'])

In [100]:
# z_org['num_probe_limit'][:] = 100
# z_trn['num_probe_limit'][:] = 100
# z_tst['num_probe_limit'][:] = 100

In [101]:
z_org['num_probe_limit'][:], z_trn['num_probe_limit'][:], z_tst['num_probe_limit'][:]

(array([100, 100, 100, ..., 100, 100, 100]),
 array([100, 100, 100, ..., 100, 100, 100]),
 array([100, 100, 100, ..., 100, 100, 100]))

In [23]:
import torch

In [25]:
torch.tensor(0).item()

0